In [15]:
#!/usr/local/bin/env python
from __future__ import division
import time
from copy import deepcopy
import numpy as np
from numpy.random import random_sample as rs
from scipy.stats.mstats import mquantiles as mq
from radd import fit, models, vis

ImportError: cannot import name fit

In [3]:
data = redf_store['redata']
binits = { k: v for k, v in redf_store['rebsl_boot_popt'].items() }
binits['ssv'] = -abs(binits['ssv'])
pinits = { k: v for k, v in redf_store['repnl_boot_popt'].items() }

avg_inits = { k: np.mean([binits[k], pinits[k]]) for k in binits.keys() }
if 'ssd' in avg_inits.keys(): del avg_inits['ssd']
      
      
data = redf_store['redata']
inits = dict(deepcopy(avg_inits))
inits['xb']=2
del inits['z']

In [4]:
inits

{'a': 0.44409409206000017,
 'ssv': -0.94733930097466645,
 'tr': 0.30487069814466666,
 'v': 1.0917075579863329,
 'xb': 2}

In [101]:
m = build.Model(data=data, inits=inits, kind='xirace', fit_on='average', depends_on={'v': 'Cond'})


      Model is prepared to fit on average xirace data, allowing
      Drift-Rate to vary across levels of Cond (bsl, pnl) 

      Optimize On, Garth 


In [118]:
m.set_fitparams()
toff = .05
simm = models.Simulator(m)
p = simm.vectorize_params(m.inits)
Pg, Tg = simm.__update_go_process__(p)
Ps, Ts = simm.__update_stop_process__(p)
            
toff = np.ceil((m.tb-(m.ssd + toff))/.0005).astype(int)

# a/tr/v Bias: ALL CONDITIONS, ALL SSD
DVg = np.cumsum(np.where((rs((m.ncond, simm.ntot, Tg.max())).T<Pg), simm.dx,-simm.dx).T, axis=2)
#np.where(DVg[:, :simm.nss, itoff:].T<a, axis=
x=np.array([np.where((DVg[:, simm.nss: itoff:].max(axis=2)<.3), -2, 1) for itoff in toff])
x.shape

(5, 2, 0)

In [172]:
DVg.shape

(2, 10000, 698)

In [180]:
np.where(DVg.reshape(simm.nssd*simm.ncond*(simm.ntot)/simm.nssd, Tg.max())<latch, 1, -2)

ValueError: operands could not be broadcast together with shapes (20000,698) (5,2000) 

In [159]:
grt = (p['tr']+(np.where((DVg.max(axis=2).T>=p['a']*.4).T, np.argmax((DVg.T>=p['a']*.4).T,axis=2)*simm.dt, np.nan).T)).T
            

In [167]:
latch.shape

(5, 2000)

In [165]:
simm.ntot/simm.nssd

2000.0

In [170]:
grt*np.where(grt.reshape(simm.ncond, simm.nssd, simm.ntot/simm.nssd)<latch, 1, 999).reshape(grt.shape)


array([[  4.63906327e+02,   4.08961327e+02,   4.52917327e+02, ...,
          4.53370698e-01,   5.20370698e-01,   4.78370698e-01],
       [  4.59910327e+02,   4.50919327e+02,   4.80889327e+02, ...,
          5.26370698e-01,   4.81370698e-01,   5.36833327e+02]])

In [128]:
DVg[:, :, 500:].max(axis=2)

array([[ 0.40249224,  0.45839394,  0.43156112, ...,  0.40249224,
         0.30410524,  0.32646592],
       [ 0.41143651,  0.41143651,  0.3421184 , ...,  0.35106267,
         0.41814471,  0.34659054]])

In [129]:
rt = (p['tr']+(np.where((DVg.max(axis=2).T>=p['a']).T, np.argmax((DVg.T>=p['a']).T,axis=2)*.0005, 999).T)).T

In [150]:
from scipy.stats.distributions import norm
x = norm.pdf(.15, .05)
#nrvs = self.nss*self.ncond
#rvs_shape = (self.nssd, nrvs/self.nssd)
#latch = (self.ssd + norm.rvs(stmu, stsig, nrvs).reshape(rvs_shape).T).T

nrvs = simm.nss*simm.ncond
rvs_shape = (simm.nssd, nrvs/simm.nssd)
latch = (simm.ssd + norm.rvs(.15, .01, nrvs).reshape(rvs_shape).T).T

(5, 1000)

In [79]:

#[np.ix_(toff)]=0
for itoff in toff:
      DVg[:, :simm.nss, itoff:]*-1=0

array([ 0.00223607, -0.00223607,  0.00223607, ...,  0.44274146,
        0.29963311,  0.43379719])

In [46]:
DVg[np.ix_([0,0,0], [698])].shape

(3, 1, 698)

In [29]:
pd.DataFrame(DVg)

ValueError: Must pass 2-d input

In [27]:
DVg[:, :, :]

IndexError: invalid slice

In [11]:
dirpath = pth + 'ELife/TESTS/'
if not os.path.isdir(dirpath):
      os.mkdir(dirpath)
os.chdir(dirpath)

data = prodf_store['prodata']
inits = dict(deepcopy(ttheta))
vinits = dict(deepcopy(vtheta))
tinits = dict(deepcopy(ttheta))

inits['a']=np.mean([tinits['a'], vinits['a']])
inits['z']=np.mean([tinits['z'], vinits['z']])
inits['v'] = np.mean(vopt.values())
inits['tr'] = np.mean(topt.values())
#inits['tr'] = np.mean(topt.values())
inits['xb']=2
#inits['si']=.01

In [13]:
m = build.Model(data=data, inits=inits, kind='xpro', fit_on='average', depends_on={'xb':'pGo'})

NameError: global name 'array' is not defined

In [1]:
m.optimize(ntrials=10000, ftol=1.e-10, xtol=1.e-10, maxfev=1000)
m.fitinfo

NameError: name 'm' is not defined

In [2]:
#yhat = m.simulate()
vis.plot_fits(m.avg_y, m.simulate(), plot_acc=True, kind='pro', save=True, savestr='dynamic_drift_pro')

NameError: name 'vis' is not defined